# 4. Model Building and Evaluation (Team-Level)

In [ ]:
# Model training
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
df = pd.read_excel('MPL_DATA_SEASON15.xlsx')
team_df = df[df['Team'].notna()].copy()
team_df['is_champion'] = (team_df['Rank'] <= 2).astype(int)
features = ['Match point', 'Net Game Win', 'Kills', 'Deaths', 'Assists', 'Gold', 'Damage', 'Lord Kills', 'Tortoise Kills', 'Tower Destroy']
X = team_df[features]
y = team_df['is_champion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
joblib.dump(model, 'team_model.pkl')

In [ ]:
# Evaluation
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

![Feature Importance](feature_importance_chart.png)

![Confusion Matrix](confusion_matrix_chart.png)

# 5. Model Deployment (Prototype)
- A Streamlit interface is provided in `app.py`
- You can deploy it using Streamlit Cloud or run locally via `streamlit run app.py`.


In [ ]:
# Streamlit Code Snippet
import streamlit as st
import pandas as pd
import numpy as np
import joblib

model = joblib.load("team_model.pkl")

st.title("🏆 MPL Team Champion Predictor")
st.write("Input team stats below to estimate if the team could win MPL S15")

inputs = {
    'Match point': st.number_input("Match Point", 0, 50, 20),
    'Net Game Win': st.number_input("Net Game Win", -10, 20, 5),
    'Kills': st.number_input("Kills", 0, 1000, 300),
    'Deaths': st.number_input("Deaths", 0, 1000, 250),
    'Assists': st.number_input("Assists", 0, 2000, 600),
    'Gold': st.number_input("Gold", 0, 1000000, 500000),
    'Damage': st.number_input("Damage", 0, 1000000, 450000),
    'Lord Kills': st.number_input("Lord Kills", 0, 20, 5),
    'Tortoise Kills': st.number_input("Tortoise Kills", 0, 30, 10),
    'Tower Destroy': st.number_input("Tower Destroy", 0, 50, 20),
}

input_df = pd.DataFrame([inputs])
prediction = model.predict(input_df)[0]
prob = model.predict_proba(input_df)[0][1]

st.markdown(f"### 🔮 Prediction: {'🏆 Likely Champion' if prediction == 1 else '❌ Unlikely to Win'}")
st.markdown(f"Confidence: **{prob*100:.2f}%**")
